# Problem Set #8

## Tc2006 Programming Languages

April 26, 2021.

_Authors of this notebook’s solution:_

- _Student ID and Name:_
- _Student ID and Name:_
- _Student ID and Name:_

**Instructions:**  Modify the Lisp metacircular evaluator code built in class in order to solve the following problems. Make sure all the unit tests are passed.

In [1]:
;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

## Main Interpreter Code

In [2]:
(import 'clojure.lang.IFn)

(declare $eval)

(defrecord Closure [env params body]
  IFn ; Implement the clojure.lang.IFn interface
  (applyTo [self args]
    (let [extended-env (merge @env (zipmap params args))]
      ($eval body extended-env))))

(defn make-closure
  [env params body]
  (->Closure (atom env) params body))

(defn set-closure-env!
  [label-name closure]
  (swap! (:env closure)
         #(assoc % label-name closure)))

(defn $eval
  [expr env]
  
  (cond
    
    ; Check for variable reference
    (symbol? expr)
    (if (contains? env expr)
      (env expr)
      (throw (RuntimeException. (str "Unbound variable: " expr))))
    
    ; Check for special forms
    (list? expr)
    (case (first expr)
      
      nil
      ()
      
      QUOTE
      (second expr)
      
      IF
      (let [[_ condition then-part else-part] expr]
        (if ($eval condition env)
          ($eval then-part env)
          ($eval else-part env)))
      
      LAMBDA
      (let [[_ params body] expr]
        (make-closure env params body))
      
      LABEL
      (let [[_ label-name label-expr] expr
            closure ($eval label-expr env)]
        (set-closure-env! label-name closure)
        closure)
      
      DO
      (if-let [args (next expr)]
        (loop [args args]
          (let [result ($eval (first args) env)]
            (if (= 1 (count args))
              result
              (recur (next args)))))
        nil)
      
      LET
      ;(let [[_ [var-name var-expr] body] expr]
      ;  ($eval body
      ;         (assoc env var-name ($eval var-expr env))))
      (let [[_ [var-name var-expr] body] expr]
        ($eval (list (list 'LAMBDA (list var-name) body) var-expr) env))
      
      ; Ordinary function application
      (let [[fun & args] (map #($eval % env) expr)]
        (apply fun args)))
    
    ; Anything that is not symbol or a list evals to itself
    :else
    expr))

#'user/$eval

## Unit Tests for the Main Interpreter

In [3]:
;;; Unit tests for variable references
(is (= 42 ($eval 'X '{W 5, X 42})))
(try
  ($eval 'Y '{W 5, X 42})
  (is false)
  (catch RuntimeException e (is true)))

true

In [4]:
;;; Unit tests for anynthing not a symbol nor list
(is (= 42 ($eval 42 {})))
(is (= true ($eval true ($eval true {}))))
(is (= "hello" ($eval "hello" ($eval true {}))))

true

In [5]:
;;; Unit tests for special form QUOTE
(is (= () ($eval () {})))
(is (= '(1 2 3) ($eval '(QUOTE (1 2 3)) {})))
(is (= 'X ($eval '(QUOTE X) {})))

true

In [6]:
;;; Unit tests for special form IF
(is (= 2 ($eval '(IF 1 2 3) {})))
(is (= 3 ($eval '(IF false 2 3) {})))
(is (= 15 ($eval '(IF A B C) '{A 7, B 15, C 42})))
(is (= 42 ($eval '(IF A B C) '{A false, B 15, C 42})))

true

In [7]:
;;; Unit tests for function application
(is (= 6 ($eval '(PLUS 1 2 3) {'PLUS +})))
(is (= 28 ($eval '(TIMES (PLUS 2 2) (MINUS 10 3)) {'PLUS +, 'TIMES *, 'MINUS -})))
(is (= 28 ($eval '(* (+ 2 2) (- 10 3)) {'+ +, '* *, '- -})))
(is (= 0 ($eval '(+) {'+ +})))
(is (= '(nil) ($eval '(f nil) {'f list})))

true

In [8]:
;;; Unit tests for special form LAMBDA
(is (= 6 ($eval '((LAMBDA (X) (+ X 1)) 5) {'+ +})))
(is (= 15($eval '((LAMBDA (X Y)
                    (* X (+ Y 1)))
                  3
                  4)
                {'+ +, '* *})))
(is (= 42 ($eval '((LAMBDA () 42)) {})))
(is (= 8 ($eval '(((LAMBDA (X)
                     (LAMBDA (Y) 
                       (+ X Y)))
                   5)
                  3)
                {'+ +})))
(is (= 8 ($eval '((LAMBDA (F X) 
                    (F (F (F X))))
                  (LAMBDA (X) (* X 2))
                  1)
                {'* *})))

true

In [9]:
;;; Unit tests for special form LABEL
(is (= '(A A B B C C D D) 
       ($eval '((LABEL DUP (LAMBDA (X)
                             (IF (NULL? X)
                               ()
                               (CONS (CAR X)
                                     (CONS (CAR X)
                                           (DUP (CDR X)))))))
                (QUOTE (A B C D)))
              {'NULL? empty?
               'CONS cons
               'CAR first
               'CDR rest})))
(is (= 120 ($eval '((LABEL FACT (LAMBDA (N)
                                  (IF (ZERO? N)
                                    1
                                    (* N (FACT (- N 1))))))
                    5)
                  {'* *
                   'ZERO? zero?
                   '- -})))

true

---

## Problem 1

Add to the Lisp interpreter the `DO` special form, which has the following syntax: 

$$
\texttt{(DO } \mathit{exp}_1 \; \mathit{exp}_2 \; \cdots \; \mathit{exp}_n \texttt{)}
$$

where every $\mathit{exp}_i$ is an expression. This construct behaves like its Clojure counterpart of the same name: it evaluates each expression in order from left to right (most likely for their side effects, such as printing), and returns the result of $\mathit{exp}_n$ (the last expression) or `nil` if there are no expressions.

Example:

    ($eval '(DO (PRN (+ -2 (+ 1 2)))
                (PRN (+ 1 1))
                (PRN 3)
                (+ 2 2))
           {'+ +
            'PRN prn})
    => 4
   
Output:

    1
    2
    3

In [10]:
;;; Unit tests:
(is (nil? ($eval '(DO)
                 {})))
(is (= 3
       ($eval '(DO (+ 1 2))
              {'+ +})))
(is (= 10
       ($eval '(DO (+ 2 5)
                   (- 2 5)
                   (/ 2 5)
                   (* 2 5))
              {'+ +
               '- -
               '/ /
               '* *})))
(is (= "7-32/510"
       (with-out-str
         ($eval '(DO (PR (+ 2 5))
                     (PR (- 2 5))
                     (PR (/ 2 5))
                     (PR (* 2 5)))
                {'PR pr
                 '+ +
                 '- -
                 '/ /
                 '* *}))))
(is (= (let [nl (System/lineSeparator)]
         (str "1" nl "2" nl "3" nl))
       (with-out-str
         ($eval '(DO (PRN (+ -2 (+ 1 2)))
                     (PRN (+ 1 1))
                     (PRN 3)
                     (+ 2 2))
                {'+ +
                 'PRN prn}))))
(is (= (let [nl (System/lineSeparator)]
         (str "One potato," nl
              "two potatoes," nl
              "three potatoes," nl
              "four." nl))
       (with-out-str
         ($eval '(DO (PRINTLN "One potato,")
                     (PRINTLN "two potatoes,")
                     (PRINTLN "three potatoes,")
                     (PRINTLN "four."))
                {'PRINTLN println}))))

true

---

## Problem 2

Add to the Lisp interpreter the `LET` special form, which has the following syntax: 

$$
\texttt{(LET}\texttt{ (}\mathit{var}\texttt{ }\mathit{expr}\texttt{) } \mathit{body}\texttt{)}
$$

where $\mathit{var}$ is a symbol, while $\mathit{expr}$ and $\mathit{body}$ are expressions. This construct evaluates and returns the result of $\mathit{body}$ using an extended environment where $\mathit{var}$ is bound with the result of evaluating $\mathit{expr}$. In other words, it's equivalent to:

$$
\texttt{((LAMBDA}\texttt{ (}\mathit{var}\texttt{) }\mathit{body}\texttt{) } \mathit{expr}\texttt{)}
$$

Examples:

    ($eval '(LET (x 6)
              (* 7 x))
           {'* *})
    => 42

    ($eval '(LET (x (* 2 5))
              (LET (y (+ 1 x))
                (+ 1 (* y x))))
           {'+ +
            '* *})
    => 111

In [11]:
;;; Unit tests:
(is (= 7
      ($eval '(LET (a 7) a) {})))
(is (= 42
      ($eval '(LET (x 6)
                (* 7 x))
        {'* *})))
(is (= 111
      ($eval '(LET (x (* 2 5))
                (LET (y (+ 1 x))
                  (+ 1 (* y x))))
        {'+ +
         '* *})))
(is (= 60
      ($eval '((LAMBDA (x y)
                 (LET (y (+ 1 y))
                   (* x y)))
                10
                5)
        {'* *
         '+ +})))
(is (= '(a b c d)
      ($eval '(LET (one (QUOTE (c d)))
                (LET (two (CONS (QUOTE b) one))
                  (LET (three (CONS (QUOTE a) two))
                    three)))
        {'CONS cons})))
(is (= 8
      ($eval '(((LAMBDA (x y)
                  (LET (z (LET (r) (* x y)) r)
                    (LAMBDA (w)
                      (LET (t (+ x (LET (a y) a) z w))
                        (LET (t t) t))))) 1 2) 3)
        {'* *
         '+ +})))

true

---

## Problem 3

Add to the Lisp interpreter the `DOTIMES` special form, which has the following syntax: 

$$
\texttt{(DOTIMES}\texttt{ (}\mathit{var}\texttt{ }\mathit{count}\texttt{) } \mathit{body}\texttt{)}
$$

where $\mathit{var}$ is a symbol, while $\mathit{count}$ and $\mathit{body}$ are expressions. This construct executes $\mathit{body}$ (which must perform some side effect operation, typically printing) once for each integer from 0 (inclusive) to $\mathit{count}$ (exclusive), while binding the variable $\mathit{var}$ to the integer for the current iteration. This special form always returns `nil`. 

Example:

    ($eval '(DOTIMES (i (+ 2 2))
              (PRINTLN "Line" i))
           {'PRINTLN println, '+ +})
    
    ($eval '(DOTIMES (x 10)
              (PR x))
           {'PR pr})
   
Output:

    Line 0
    Line 1
    Line 2
    Line 3

    0123456789

In [ ]:
;;; Unit tests:
(is (= ""
       (with-out-str ($eval '(DOTIMES (x 0)
                               (PRINTLN x))
                            {'PRINTLN println}))))
(is (= "0123456789"
       (with-out-str ($eval '(DOTIMES (x 10)
                               (PR x))
                            {'PR pr}))))
(is (= (let [nl (System/lineSeparator)]
         (str "Line 0" nl "Line 1" nl "Line 2" nl "Line 3" nl))
       (with-out-str ($eval '(DOTIMES (i (+ 2 2))
                               (PRINTLN "Line" i))
                            {'PRINTLN println, '+ +}))))
(is (= "1-4-9-16-25-36-49-64-81-100-"
       (with-out-str ($eval '(DOTIMES (some-var (* 2 5))
                               (PRINTF "%d-"
                                       ((LAMBDA (x) (* x x))
                                        (INC some-var))))
                            {'PRINTF printf, '* *, 'INC inc}))))
(is (= (str "**************************************************"
            "**************************************************")
       (with-out-str ($eval '(DOTIMES (mxyzptlk (* 2 2 5 5))
                               (PRINT "*"))
                            {'PRINT print, '* *}))))

---

## Problem 4

Add to the Lisp interpreter the `COND` special form, which has the following syntax: 

$$
\texttt{(COND }
  \mathit{test}_1 \; \mathit{exp}_1 \;
  \mathit{test}_2 \; \mathit{exp}_2 \;
  \cdots \;
  \mathit{test}_n \; \mathit{exp}_n
  \texttt{)}
$$

where every $\mathit{test}_i$ and $\mathit{exp}_i$ is an expression. This construct, like its Clojure counterpart of the same name, takes a sequence of $\mathit{test}$/$\mathit{exp}$ pairs. It evaluates each $\mathit{test}$ one at a time. If a $\mathit{test}$ returns logical true (any value not equal to `nil` nor `false`), `COND` evaluates and returns the value of the corresponding $\mathit{exp}$ and doesn't evaluate any of the other $\mathit{test}$s or $\mathit{exp}$s. If there are no tests that evaluate to true, `COND` returns `nil`.

You can alternatively think of `COND` as being equivalent to: 

$$
\texttt{(if } \mathit{test}_1 \; \mathit{exp}_1 \;
\texttt{(if } \mathit{test}_2 \; \mathit{exp}_2 \;
\cdots \;
\texttt{(if } \mathit{text}_n \; \mathit{exp}_n \; \texttt{nil)}
\cdots
\texttt{))}
$$

Example:

    ($eval '(COND
              (= x 1) (QUOTE one)
              (= x 2) (QUOTE two)
              (= x 3) (QUOTE three)
              (= x 4) (QUOTE four)
              true    (QUOTE other))
              {'x 3
               '= =})
    => three

In [ ]:
;;; Unit tests:
(is (nil? ($eval '(COND)
                 {})))
(is (= 2
       ($eval '(COND
                 1 2)
              {})))
(is (nil? ($eval '(COND
                    false 2)
                 {})))
(is (= 'three
       ($eval '(COND
                 (= x 1) (QUOTE one)
                 (= x 2) (QUOTE two)
                 (= x 3) (QUOTE three)
                 (= x 4) (QUOTE four)
                 true    (QUOTE other))
              {'x 3
               '= =})))
(is (= 'other
       ($eval '(COND
                 (= x 1) (QUOTE one)
                 (= x 2) (QUOTE two)
                 (= x 3) (QUOTE three)
                 (= x 4) (QUOTE four)
                 true    (QUOTE other))
              {'x 5
               '= =})))
(is (nil? ($eval '(COND
                    (= x 1) (QUOTE one)
                    (= x 2) (QUOTE two)
                    (= x 3) (QUOTE three)
                    (= x 4) (QUOTE four))
                 {'x 5
                  '= =})))
(is (= "Lannister"
       (with-out-str
         ($eval '(COND
                   (< 4 got) (PRINT "Targaryen")
                   (< 3 got) (PRINT "Stark")
                   (< 2 got) (PRINT "Tully")
                   (< 1 got) (PRINT "Lannister")
                   true      (PRINT "Tyrell"))
                {'got 2
                 '< <
                 'PRINT print}))))